In [ ]:
import pygplates
import sys, os
import glob
import PlateFrameRasters
from surface_plotting import create_model_file_list_by_time
from model_config import model_defs 

wencao = False

ModelNum = 'gld428_m21'
gen_delta = False
#gen_delta = True
continent_only = True #set to true to generate continent_only grids
#continent_only = False
with_ocean = True #set to true to generate grids with data in the oceans

root_path = './'
ModelPath = root_path + '/models/'

start_time = -1
end_time = 1e7
#if wencao:
#    start_time = 249
#    end_time = 410

model_time_list = create_model_file_list_by_time(ModelPath,ModelNum,Frame='Mantle',Res=None)
# print(model_time_list)
model_time_list = [x for x in model_time_list if end_time>=x[0]>=start_time]

multipoint_feature_collection_name = root_path + model_defs[ModelNum]['multipoint']

if 'rotations' in model_defs[ModelNum]:
    input_rotation_filename = [root_path + x for x in model_defs[ModelNum]['rotations']]
elif 'rotation_dir' in model_defs[ModelNum]:
    input_rotation_filename = glob.glob(root_path+model_defs[ModelNum]['rotation_dir']+'/*.rot')

#print input_rotation_filename

raster_times = [tmp[0] for tmp in model_time_list]
raster_filenames = [tmp[1] for tmp in model_time_list]
print(raster_filenames)

rotation_model = pygplates.RotationModel(input_rotation_filename)

file_registry = pygplates.FeatureCollectionFileFormatRegistry()
print('Reading multipoint data: '+multipoint_feature_collection_name)
multipoint_feature_collection = file_registry.read(multipoint_feature_collection_name)
original_multipoint_feature_collection = file_registry.read(multipoint_feature_collection_name)

# Optionally exclude multipoint features with appearance age younger than threshold
min_time = 200

os.system('rm -rf '+ModelPath + ModelNum +'/Dynamic_topography/dist/')
os.system('rm -rf '+ModelPath + ModelNum +'/Dynamic_topography/dist_deltas/')
out_dir = ModelPath + ModelNum +'/Dynamic_topography/dist/PlateFrame/'
if not os.path.isdir(out_dir):
    os.system('mkdir --parents ' + out_dir)

if with_ocean:
    selected_features = multipoint_feature_collection.get(
        lambda feature: feature.get_reconstruction_plate_id()!=0,
        pygplates.FeatureReturn.all)
    
    multipoint_feature_collection = pygplates.FeatureCollection(selected_features)

    # To Get Plate Frame Grids of Dynamic Topography
    outfile_stem = out_dir
    PlateFrameRasters.GeneratePlateReferenceFramesXYZ(
        rotation_model, 
        raster_times, 
        raster_filenames, 
        multipoint_feature_collection,
        outfile_stem)

    
if continent_only:
    selected_features = multipoint_feature_collection.get(
        lambda feature: feature.get_valid_time()[0]>min_time and feature.get_reconstruction_plate_id()!=0, 
        pygplates.FeatureReturn.all)

    multipoint_feature_collection = pygplates.FeatureCollection(selected_features)


    # To Get Plate Frame Grids of Dynamic Topography
    outfile_stem = out_dir+'Masked_'
    
    #if not os.path.isdir(outfile_stem) and not outfile_stem.endswith('_'):
        #os.system('mkdir --parents ' + outfile_stem)

    PlateFrameRasters.GeneratePlateReferenceFramesXYZ(
        rotation_model, 
        raster_times, 
        raster_filenames, 
        multipoint_feature_collection,
        outfile_stem)

for f in os.listdir(out_dir):
    os.system('mv '+out_dir+f+' ' +out_dir+f.replace('PlateFrameGrid',''))
    
if gen_delta:
    # To Get Plate Frame Grids of RATE OF CHANGE of Dynamic Topography 
    #outfile_stem = './images/%s_' % ModelNum
    out_dir = ModelPath + ModelNum +'/Dynamic_topography/dist_deltas/PlateFrame/'
    if not os.path.isdir(out_dir):
        os.system('mkdir --parents ' + out_dir)      

    #without oceans
    selected_features = original_multipoint_feature_collection.get(
        lambda feature: feature.get_valid_time()[0]>min_time and feature.get_reconstruction_plate_id()!=0, 
        pygplates.FeatureReturn.all)

    multipoint_feature_collection = pygplates.FeatureCollection(selected_features)

    outfile_stem = out_dir+'Masked_'
    print(outfile_stem)
    PlateFrameRasters.GenerateReconstructedDeltasXYZ(rotation_model, 
                                                     raster_times, 
                                                     raster_filenames, 
                                                     multipoint_feature_collection, 
                                                     outfile_stem,
                                                     force_plate_frame=True)
    
    #with oceans
    selected_features = original_multipoint_feature_collection.get(
        lambda feature: feature.get_reconstruction_plate_id()!=0,
        pygplates.FeatureReturn.all)
    
    multipoint_feature_collection = pygplates.FeatureCollection(selected_features)

    outfile_stem = out_dir
    print(outfile_stem)
    PlateFrameRasters.GenerateReconstructedDeltasXYZ(rotation_model, 
                                                     raster_times, 
                                                     raster_filenames, 
                                                     multipoint_feature_collection, 
                                                     outfile_stem,
                                                     force_plate_frame=True)

    for f in os.listdir(out_dir):
        os.system('mv '+out_dir+f+' ' +out_dir+f.replace('PlateFrameDelta',''))

    # To Get Plate Frame Grids of RATE OF CHANGE of Dynamic Topography 
    #outfile_stem = './images/%s_' % ModelNum
    outfile_stem = ModelPath + ModelNum +'/Dynamic_topography/dist_deltas/MantleFrame/'
    if not os.path.isdir(outfile_stem):
        os.system('mkdir --parents ' + outfile_stem)
    print(outfile_stem)
    PlateFrameRasters.GenerateReconstructedDeltasXYZ(rotation_model, 
                                                     raster_times, 
                                                     raster_filenames, 
                                                     multipoint_feature_collection, 
                                                     outfile_stem,
                                                     force_plate_frame=False)
    for f in os.listdir(outfile_stem):
        os.system('mv '+outfile_stem+f+' ' +outfile_stem+f.replace('ReconstructedDelta',''))

print('done')


In [ ]:
for f in os.listdir(out_dir):
    print(out_dir+f)

#### The code cell below is for test purpose only. It will draw a netcdf grid.
Change the __InputGridFile__ variable to the file you want to see.

In [ ]:
import numpy as np
from netCDF4 import Dataset
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
%matplotlib inline

# @_@ LOOK HERE 
#change this variable to the file you want to see.
InputGridFile = './models/gld428/Dynamic_topography/dist/PlateFrame/40.00.nc'

def cmap_discretize(cmap, N):
    #http://wiki.scipy.org/Cookbook/Matplotlib/ColormapTransformations
    """Return a discrete colormap from the continuous colormap cmap.     
    cmap: colormap instance, eg. cm.jet. 
    N: number of colors.
    
    Example
    x = resize(arange(100), (5,100))
    djet = cmap_discretize(cm.jet, 5)
    imshow(x, cmap=djet)
    """
    import matplotlib
    
    if type(cmap) == str:
        cmap = get_cmap(cmap)
    colors_i = np.concatenate((np.linspace(0, 1., N), (0.,0.,0.,0.)))
    colors_rgba = cmap(colors_i)
    indices = np.linspace(0, 1., N+1)
    cdict = {}
    for ki,key in enumerate(('red','green','blue')):
        cdict[key] = [ (indices[i], colors_rgba[i-1,ki], colors_rgba[i,ki]) for i in range(N+1) ]
    
    # Return colormap object.
    return matplotlib.colors.LinearSegmentedColormap(cmap.name + "_%d"%N, cdict, 1024)


data = Dataset(InputGridFile,'r')
gridX,gridY = np.meshgrid(data.variables['lon'][:],data.variables['lat'][:])
gridZ = data.variables['z'][:]

# create figure and axes instances
fig = plt.figure(figsize=(16,12),dpi=300,frameon=False)
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
cm = cmap_discretize(plt.cm.RdBu_r, 30)
cb=ax.imshow(gridZ, origin='lower', transform=ccrs.PlateCarree(),
          extent=[-180, 180, -90, 90], cmap=cm)

fig.colorbar(cb, shrink=0.5, label='Dynamic Topography (m)', orientation="horizontal", pad=0.05)
plt.title(InputGridFile)
plt.show()